## Domain adaptation for BERT
based on https://github.com/huggingface/notebooks/blob/main/examples/language_modeling.ipynb this notebook can be found on kaggle together with the dataset from domain-adaptation.ipynb

You will need both:

- huggingface tokens with write permissions 
- wandb token

In [1]:
from transformers import AutoTokenizer
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForMaskedLM

import pandas as pd

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from huggingface_hub import notebook_login

notebook_login()

# Upload own data

We will use the wiki text ( from Food.csv ) from the fooddb 

In [3]:
domain_adapatation_text = pd.read_csv("/kaggle/input/domain-adaptation-on-foodb/text_data_for_domain_adaptation.csv")
domain_adapatation_text.dropna()
domain_adapatation_text.head()

,text
0,Angelica is a genus of about 60 species of tal...
1,Savoy cabbage (Brassica oleracea convar. capit...
2,Tilia tomentosa (Silver Lime in the UK and Sil...
3,"The kiwifruit, often shortened to kiwi in many..."
4,Allium haematochiton is a species of wild onio...


**We will try to fune tune an llm on the masked descriptions**

In [4]:
from datasets import Dataset, DatasetDict

In [5]:
ds = Dataset.from_pandas(domain_adapatation_text)

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [6]:
ds

Dataset({
    features: ['text'],
    num_rows: 945
})

In [7]:
train_testvalid = ds.train_test_split(test_size=0.2)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
datasets = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

In [8]:
# inspect the dataset
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(datasets["train"])

,text
0,"Myrica is a genus of about 35?50 species of small trees and shrubs in the family Myricaceae, order Fagales. The genus has a wide distribution, including Africa, Asia, Europe, North America and South America, and missing only from Australia. Some botanists split the genus into two genera on the basis of the catkin and fruit structure, restricting Myrica to a few species, and treating the others in Morella. The fruit is a small drupe, usually with a wax coating.\nCommon names include Bayberry, Bay-rum tree, Candleberry, Sweet Gale, and Wax-myrtle. The generic name was derived from the Greek word ?????? (myrike), meaning ""fragrance.\n"
1,"Vegetarian cuisine refers to food that meets vegetarian standards by not including meat and animal tissue products. For lacto-ovo vegetarianism (the most common type of vegetarianism in the Western world), eggs and dairy products such as milk and cheese are permitted. For lacto vegetarianism, the earliest known type of vegetarianism, dairy products such as milk and cheese are permitted. The strictest forms of vegetarianism are veganism and fruitarianism, which exclude all animal products, including dairy products as well as honey, and even some refined sugars if filtered and whitened with bone char. Vegetarian foods can be classified into several different types: Traditional foods that have always been vegetarian Soy products including tofu and tempeh which are common protein sources Textured vegetable protein (TVP), made from defatted soy flour, often included in chili and burger recipes in place of ground meat Meat analogues, which mimic the taste, texture, and appearance of meat and are often used in recipes that traditionally contained meat. Vegans may also use analogues for eggs and dairy products"
2,"Casein is the name for a family of related phosphoprotein proteins (alphaS1, alphaS2, beta, kappa). These proteins are commonly found in mammalian milk, making up 80% of the proteins in cow milk and between 60% and 65% of the proteins in human milk. Casein has a wide variety of uses, from being a major component of cheese, to use as a food additive, to a binder for safety matches. As a food source, casein supplies amino acids; carbohydrates; and two inorganic elements, calcium and phosphorus. [Wikipedia]"
3,"<i>Acipenser</i> is a genus of sturgeons. With 17 species, many of which are threatened, it is the largest genus in the order Acipenseriformes. (Wikipedia)"
4,"The walrus (Odobenus rosmarus) is a large flippered marine mammal with a discontinuous distribution about the North Pole in the Arctic Ocean and subarctic seas of the Northern Hemisphere. The walrus is the only living species in the Odobenidae family and Odobenus genus. This species is subdivided into three subspecies: the Atlantic walrus (O. r. rosmarus) which lives in the Atlantic Ocean, the Pacific walrus (O. r. divergens) which lives in the Pacific Ocean, and O. r. laptevi, which lives in the Laptev Sea of the Arctic Ocean.\nAdult walruses are easily recognized by their prominent tusks, whiskers, and bulkiness. Adult males in the Pacific can weigh more than 2,000 kg (4,400 lb) and, among pinnipeds, are exceeded in size only by the two species of elephant seals. Walruses live mostly in shallow waters above the continental shelves, spending significant amounts of their lives on the sea ice looking for benthic bivalve mollusks to eat. Walruses are relatively long-lived, social animals, and they are considered to be a ""keystone species"" in the Arctic marine regions."
5,"Diospyros kaki, better known as the Japanese Persimmon, Kaki Persimmon (kaki [?]) or Asian Persimmon in North America, is the most widely cultivated species of the Diospyros genus. Although its first published botanical description was not until 1780, the kaki is also among the oldest plants in cultivation, known for its use in China for more than 2000 years. In some rural Chinese communities, the kaki fruit is seen as having a great mystical power that c

In [9]:
model_checkpoint = 'bert-base-uncased' # later try 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [10]:
from functools import partial

def tokenize_function(tokenizer, examples):
    return tokenizer(examples['text'])

partial_tokenize_function = partial(tokenize_function, tokenizer)

In [11]:
tokenized_datasets = datasets.map(partial_tokenize_function, batched=True, num_proc=4, remove_columns=['text'])

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (574 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (551 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (643 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (532 > 512). Running this sequence through the model will result in indexing errors


...

Token indices sequence length is longer than the specified maximum sequence length for this model (789 > 512). Running this sequence through the model will result in indexing errors


#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [12]:
# block_size = tokenizer.model_max_length
block_size = 128
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [13]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [14]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-dietdupe-foodb-desc",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

In [16]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

In [19]:
trainer.train()

wandb: Currently logged in as: zukowskanina1 (ja-nina). Use `wandb login --relogin` to force relogin


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,1.866525
2,No log,1.752035
3,No log,1.741570


TrainOutput(global_step=375, training_loss=1.9016984049479166, metrics={'train_runtime': 94.637, 'train_samples_per_second': 31.637, 'train_steps_per_second': 3.963, 'total_flos': 197008802380800.0, 'train_loss': 1.9016984049479166, 'epoch': 3.0})

In [20]:
import math

In [21]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 5.73


In [23]:
trainer.push_to_hub()

'https://huggingface.co/ja-nina/bert-base-uncased-dietdupe-foodb-desc/tree/main/'